Creo dos estructuras de datos que relacionen términos con CUIs y CUIs con términos.

In [1]:
import pandas as pd
import time

mrconso_path = '../.data/META/MRCONSO.RRF'
interest_columns = ['CUI', 'LAT', 'STR', 'SAB']


start_time = time.time()
try:
    mrconso_df = pd.read_csv(
        mrconso_path,
        sep = '|',
        header=None,
        names=interest_columns,
        dtype=str,
        encoding='utf-8'
    )
except FileNotFoundError:
    print(f"ERROR: File not found at {mrconso_path}. Please check your path.")
    
except Exception as e:
    print(f"An error occurred during loading: {e}")
    
    
end_time = time.time()
print(f"Loaded MRCONSO.RRF in {end_time - start_time:.2f} seconds.")

Loaded MRCONSO.RRF in 231.62 seconds.


In [ ]:
mrconso_df.head()

NameError: name 'mrconso_df' is not defined

In [ ]:
# Load all the values in the LAT column
lat_values = mrconso_df['LAT'].unique()
print(f"Unique LAT values: {lat_values}")

# Escojemos sólo los términos en español.
esp_mrconso_df = mrconso_df[mrconso_df['LAT'] == 'ESP']
# Count the number of STR entries in MRCONSO
num_str_entries = esp_mrconso_df.shape[0]
print(f"Number of STR entries in MRCONSO: {num_str_entries}")

Unique LAT values: ['N' 'O' 'Y' 'E']
Number of STR entries in MRCONSO: 0


He cargado las columnas de MRCONSO.RRF de interés en un dataframe. Uso ahora un texto de ejemplo extraído de una de las notas clínicas:

In [ ]:
note = "CICLO 21 T-DM1Carcinoma inflamatorio bilateral de mama (G3) diagnosticado en 2007. T4N2M0 (Estadio IIIB). Triple positivo.*QT- ACT, trastuzumab y hormonoterapia. Mastectomía bilateral +LA. RT pared torácica derecha e izquierda, áreas ganglionares axilares bilaterales y cérvico claviculares. Tamoxifeno oct 2007 – noviembre 2012.- Recaída hepatica y ganglionar (oct 2014). Ca mama estadio IV HER2+ en Octubre del 2014.*El 30.10.2014 inicia Taxol semanal + trastuzumab + pertuzumab trisemanal x 7 ciclos (05.03.2015), continuando con pertuzumab + herceptin con buena respuesta.*El 24/11/2015 se realiza segmentectomía hepática VII + subsegmentectomía VI de metástasis hepática y sección de vena suprahepática derecha.*Desde entonces, pertuzumab-herceptín-enero 2017*TAC enero 2017: adenopatía anterosup mediastinica*PERTU -TRASTU + LETROZOL*Progresión a nivel abdominal con masa pancreática con confirmación histológica.*SBRT sobre la lesión pancreatica entre el 6 y el 16/12/2019.*TDM1 desde el 26/7/2019Sospecha de ocupacion a nivel de la protesis biliar .Se ha retrasado la administraciond e qt ya mas de un mes ultima dosis administrada el 10 de Dic, mantienen BIl estable y tiene citado TAC en 2 dias .Administro tto , y cito en 1 semana para ver resultados del TAC y valorar."

Antes de pasar a buscar elementos similares en UMLS, es necesario extraer del texto entidades candidatas para ser normalizadas. No es lo mismo la entidad "Carcinoma inflamatorio bilateral de mama" que "mama". Por ello necesito hacer dos cosas: Limpiar el texto y extraer entidades.

In [ ]:
from note_cleaner import clean_note

cleaned_note = clean_note(note)
print(cleaned_note)

CICLO 21 T-DM1

Carcinoma inflamatorio bilateral de mama (G3) diagnosticado en 2007. T4N2M0 (Estadio IIIB). Triple positivo.

QT-ACT, trastuzumab y hormonoterapia. Mastectomía bilateral +LA. RT pared torácica derecha e izquierda, áreas ganglionares axilares bilaterales y cérvico claviculares. Tamoxifeno octubre 2007 – noviembre 2012.

Recaída hepática y ganglionar (octubre 2014). Ca mama estadio IV HER2+ en Octubre del 2014.

El 30.10.2014 inicia Taxol semanal + trastuzumab + pertuzumab trisemanal x 7 ciclos (05.03.2015), continuando con pertuzumab + herceptin con buena respuesta.

El 24/11/2015 se realiza segmentectomía hepática VII + subsegmentectomía VI de metástasis hepática y sección de vena suprahepática derecha.

Desde entonces, pertuzumab-herceptín (enero 2017)

TAC enero 2017: adenopatía anterosup mediastinica

PERTU -TRASTU + LETROZOL

Progresión a nivel abdominal con masa pancreática con confirmación histológica.

SBRT sobre la lesión pancreática entre el 6 y el 16/12/2019.


Para obtener candidatos a normalizar necesito algún modelo encargado de elegir candidatos. Empezaré con un enfoque tradicional, usando todos los n-gramas obtenidos en el texto.

In [ ]:
import spacy


nlp = spacy.load("es_core_news_sm")
doc = nlp(cleaned_note)

# Count the time for this operation
import time
start_time = time.time()

# Have all n-grams of size 1 to 10
all_ngrams = []
for n in range(1, 11):
    ngrams = zip(*[doc[i:] for i in range(n)])
    all_ngrams.extend([' '.join([token.text for token in ngram]) for ngram in ngrams])

end_time = time.time()
print(f"N-gram extraction took {end_time - start_time:.4f} seconds.")
print(all_ngrams)

N-gram extraction took 0.0046 seconds.
['CICLO', '21', 'T-DM1', '\n\n', 'Carcinoma', 'inflamatorio', 'bilateral', 'de', 'mama', '(', 'G3', ')', 'diagnosticado', 'en', '2007', '.', 'T4N2M0', '(', 'Estadio', 'IIIB', ')', '.', 'Triple', 'positivo', '.', '\n\n', 'QT-ACT', ',', 'trastuzumab', 'y', 'hormonoterapia', '.', 'Mastectomía', 'bilateral', '+', 'LA', '.', 'RT', 'pared', 'torácica', 'derecha', 'e', 'izquierda', ',', 'áreas', 'ganglionares', 'axilares', 'bilaterales', 'y', 'cérvico', 'claviculares', '.', 'Tamoxifeno', 'octubre', '2007', '–', 'noviembre', '2012', '.', '\n\n', 'Recaída', 'hepática', 'y', 'ganglionar', '(', 'octubre', '2014', ')', '.', 'Ca', 'mama', 'estadio', 'IV', 'HER2', '+', 'en', 'Octubre', 'del', '2014', '.', '\n\n', 'El', '30.10.2014', 'inicia', 'Taxol', 'semanal', '+', 'trastuzumab', '+', 'pertuzumab', 'trisemanal', 'x', '7', 'ciclos', '(', '05.03.2015', ')', ',', 'continuando', 'con', 'pertuzumab', '+', 'herceptin', 'con', 'buena', 'respuesta', '.', '\n\n', 'El'

In [ ]:
# Check if any n-gram matches the STR column in mrconso_df
matched_cuis = set()
for ngram in all_ngrams:
    matches = mrconso_df[mrconso_df['STR'].str.lower() == ngram.lower()]
    for _, row in matches.iterrows():
        matched_cuis.add((row['CUI'], row['STR'], row['SAB']))
print("Exact matched CUIs:")
for cui, str_val, sab in matched_cuis:
    print(f"CUI: {cui}, STR: {str_val}, SAB: {sab}")

Exact matched CUIs:


In [ ]:
# Count the number of STR entries in MRCONSO
num_str_entries = mrconso_df.shape[0]
print(f"Number of STR entries in MRCONSO: {num_str_entries}")

Number of STR entries in MRCONSO: 0
